# Introduction


In this assignment, you will practice building and training Convolutional Neural Networks with Pytorch to solve computer vision tasks.  This assignment includes two sections, each involving different tasks:

(1) Image Classification. Predict image-level category labels on two historically notable image datasets: **CIFAR-10** and **MNIST**.

(2) Image Segmentation. Predict pixel-wise classification (semantic segmentation) on synthetic input images formed by superimposing MNIST images on top of CIFAR images.

You will design your own models in each section and build the entire training/testing pipeline with PyTorch. 
PyTorch provides optimized implementations of the building blocks and additional utilities, both of which will be necessary for experiments on real datasets. It is highly recommended to read the official [documentation](https://pytorch.org/docs/stable/index.html) and [examples](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) before starting your implementation. There are some APIs that you'll find useful:
[Layers](http://pytorch.org/docs/stable/nn.html),
[Activations](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity),
[Loss functions](http://pytorch.org/docs/stable/nn.html#loss-functions),
[Optimizers](http://pytorch.org/docs/stable/optim.html)

It is highly recommended to use Google Colab and run the notebook on a GPU node. Check https://colab.research.google.com/ and look for tutorials online. To use a GPU go to Runtime -> Change runtime type and select GPU. 






# (1) Image Classification

In this section, you will design and train an image classification network, which takes images as input and outputs vectors whose length equals the number of possible categories on **MNIST** and **CIFAR-10** datasets. 

You can design your models by borrowing ideas from recent architectures, e.g., ResNet, but you may not simply copy an entire existing model. 

For image classification, you can use a built-in dataset provided by [torchvision](https://pytorch.org/vision/stable/index.html), a PyTorch official extension for image tasks. 

To finish this section step by step, you need to:

* Prepare data by building a dataset and dataloader. (with [torchvision](https://pytorch.org/vision/stable/index.html))

* Implement training code (6 points) & testing code (6 points), including model saving and loading.

* Construct a model (12 points) and choose an optimizer (3 points).

* Describe what you did, any additional features you implemented, and/or any graphs you made in training and evaluating your network. Also report final test accuracy @100 epochs in a writeup: hw3.pdf (3 points)

In [1]:
import numpy as np
import os
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision
import torchvision.transforms as T


## Data Preparation:

Setup a Dataset for training and testing.

Datasets load single training examples one a time, so we practically wrap each Dataset in a DataLoader, which loads a data batch in parallel.

We provide an example for setting up a training set for MNIST, and you should complete the rest. 

In [40]:
mnist_train = torchvision.datasets.MNIST('./data', train = True, download = True, transform = T.ToTensor())
mnist_train_data_loader = torch.utils.data.DataLoader(mnist_train,
                                          batch_size=400,
                                          shuffle=True,
                                          num_workers=2)

mnist_test = torchvision.datasets.MNIST("./data", train = False, download = True, transform = T.ToTensor())
mnist_test_data_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size=400,
                                          shuffle=True,
                                          num_workers=2)
print(mnist_train)
print(mnist_test)
##########################################################################
# TODO: YOUR CODE HERE
# (1) Instantiate the train/test split for CIFAR-10 and MNIST
# (2) You can adjust the batch size and number of works for better efficiency
# (3) Remember to set shuffle=True for all training set 
##########################################################################

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()


## Design/choose your own model structure (12 points) and optimizer (3 points).
You might want to adjust the following configurations for better performance:

(1) Network architecture:
- You can borrow some ideas from existing CNN designs, e.g., ResNet where
the input from the previous layer is added to the output
https://arxiv.org/abs/1512.03385
- Note: Do not **directly copy** an entire existing network design.

(2) Architecture hyperparameters:
- Filter size, number of filters, and number of layers (depth). Make careful choices to tradeoff computational efficiency and accuracy.
- Pooling vs. Strided Convolution
- Batch normalization
- Choice of non-linear activation

(3) Choice of optimizer (e.g., SGD, Adam, Adagrad, RMSprop) and associated hyperparameters (e.g., learning rate, momentum).

In [41]:
#Basic model, feel free to customize the layout to fit your model design.

##########################################################################
# TODO: YOUR CODE HERE
# (1) Design the model for MNIST
# (2) Design the model for CIFAR-10
##########################################################################

# model with 1 convolution layer 
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet, self).__init__()
        # Should add more layers and max pool after activating
        self.convlayer1 = nn.Sequential(nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=3, stride=1, padding=1), nn.ReLU())
        self.out = nn.Linear(28*28*16, 10)


    def forward(self, x):
        x = self.convlayer1(x)
        x = x.view(x.size(0), -1)
        out = self.out(x)
        return out
mnistNet1 = MnistNet()
optimizer = optim.Adam(mnistNet1.parameters(), lr = 0.001, betas=(0.9,0.999))
print(optimizer)


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


## Training (6 points)

Train a model on the given dataset using the PyTorch Module API.

Inputs:
- loader_train: The loader from which train samples will be drawn from.
- loader_test: The loader from which test samples will be drawn from.
- model: A PyTorch Module giving the model to train.
- optimizer: An Optimizer object we will use to train the model.
- epochs: (Optional) A Python integer giving the number of epochs to train for.

Returns: Nothing, but prints model accuracies during training.

In [42]:
def train(loader_train, model, optimizer, epochs=100):

    # Working locally on laptop and desktop, sometimes no gpu available. Device is gpu if gpu is available, cpu otherwise
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    
    # TODO Improvement idea: test on loader_test and stop if early stoppage
    for e in range(epochs):
        model.train()
        for t, (x, y) in enumerate(loader_train):
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            ##########################################################################
            # TODO: YOUR CODE HERE
            # (1) move data to GPU DONE
            # (2) forward and get loss
            # (3) zero out all of the gradients for the variables which the optimizer
            # will update.
            # (4) the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            # (5) update the parameters of the model using the gradients
            # computed by the backwards pass.
            ##########################################################################
            if t % 100 == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
train(mnist_train_data_loader, mnistNet1, optimizer, epochs= 1)

cpu
Epoch 0, Iteration 0, loss = 2.2983
Epoch 0, Iteration 100, loss = 0.2012


## Testing (6 points)
Test a model using the PyTorch Module API.

Inputs:
- loader: The loader from which test samples will be drawn from.
- model: A PyTorch Module giving the model to test.

Returns: Nothing, but prints model accuracies during training.

In [43]:
def test(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval() # set model to evaluation mode
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            num_samples += x.size()[0]
           # print(f"labels (y) are: {y}")
            outputs = torch.argmax(model(x),1)
            num_correct += (outputs == y).sum()
            print(f"num_samples: {num_samples}")
            print(f"num_correct: {num_correct}")
           # print(f"ouptuts are: {outputs}")
            ##########################################################################
            # TODO: YOUR CODE HERE
            # (1) move to GPU DONE
            # (2) forward and calculate scores and predictions
            # (2) accumulate num_correct and num_samples
            ##########################################################################
    acc = num_correct / num_samples
    print('Eval %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
test(mnist_test_data_loader, mnistNet1)

num_samples: 400
num_correct: 376
num_samples: 800
num_correct: 753
num_samples: 1200
num_correct: 1120
num_samples: 1600
num_correct: 1490
num_samples: 2000
num_correct: 1865
num_samples: 2400
num_correct: 2247
num_samples: 2800
num_correct: 2621
num_samples: 3200
num_correct: 3004
num_samples: 3600
num_correct: 3377
num_samples: 4000
num_correct: 3750
num_samples: 4400
num_correct: 4129
num_samples: 4800
num_correct: 4504
num_samples: 5200
num_correct: 4878
num_samples: 5600
num_correct: 5257
num_samples: 6000
num_correct: 5633
num_samples: 6400
num_correct: 6006
num_samples: 6800
num_correct: 6379
num_samples: 7200
num_correct: 6759
num_samples: 7600
num_correct: 7141
num_samples: 8000
num_correct: 7510
num_samples: 8400
num_correct: 7890
num_samples: 8800
num_correct: 8268
num_samples: 9200
num_correct: 8647
num_samples: 9600
num_correct: 9020
num_samples: 10000
num_correct: 9394
Eval 9394 / 10000 correct (93.94)


Describe your design details in the writeup hw3.pdf. (3 points)

Finish your model and optimizer below.

In [ ]:
model = CIFAR10_models()
optimizer = optim.SGD(model.parameters(), lr, momentum, weight_decay)
train(cifar10_loader_train, cifar10_loader_test, model, optimizer, epochs=100)

In [ ]:
model = MNIST_models()
optimizer = optim.SGD(model.parameters(), lr, momentum, weight_decay)
train(mnist_loader_train, mnist_loader_test, model, optimizer, epochs=100)